
# 03 · Synthesis Demo (Tokens + Gestures → MIDI/Audio)

**Goal:** Combine symbolic tokens (notes) and gesture dynamics to render a simple MIDI/audio demo.

👉 If you're on Colab:  
```bash
!pip install music21 mido pygame matplotlib numpy
```


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from music21 import stream, note, midi

# Example placeholder files (replace with real outputs)
tokens_file = 'examples/tokens.json'
gestures_file = 'examples/conductor_take01.npz'

print("Expecting:", tokens_file, gestures_file)


In [ ]:

import os, json

if os.path.exists(tokens_file):
    with open(tokens_file) as f:
        tokens = json.load(f)
    print("Loaded tokens:", tokens[:10] if isinstance(tokens, list) else type(tokens))
else:
    print("⚠️ No tokens found, using dummy sequence")
    tokens = [60, 62, 64, 65, 67]  # C major scale

if os.path.exists(gestures_file):
    g = np.load(gestures_file)
    dynamics = g['conf'][:len(tokens)]
    print("Loaded gesture dynamics:", dynamics.shape)
else:
    print("⚠️ No gesture file, using flat dynamics")
    dynamics = np.ones(len(tokens)) * 0.8


In [ ]:

# Build a music21 Stream with dynamics as note velocities
s = stream.Stream()
for pitch, dyn in zip(tokens, dynamics):
    n = note.Note(pitch)
    n.quarterLength = 1
    n.volume.velocity = int(dyn * 127)
    s.append(n)

# Save to MIDI
mf = midi.translate.streamToMidiFile(s)
out_midi = "examples/output_demo.mid"
mf.open(out_midi, 'wb')
mf.write()
mf.close()
print("Saved MIDI:", out_midi)


In [ ]:

plt.figure(figsize=(6,2))
plt.plot(dynamics, marker='o')
plt.title("Gesture Dynamics over Notes")
plt.xlabel("Note Index")
plt.ylabel("Velocity (0-127)")
plt.show()
